# IBM Quantum Challenge 2020 November - Exercise 3

[Notebook containing the challenge exercise](https://github.com/qiskit-community/IBMQuantumChallenge2020/blob/main/exercises/week-3/final_en.ipynb)

## Dependencies

In [1]:
from qiskit import Aer, ClassicalRegister, execute, QuantumCircuit, QuantumRegister
import numpy as np

## Helper Functions

In [2]:
def rccx(qc, a, b, c):
    """Function to implement a simplified two-control Toffoli gate.

    Parameters
    ----------
    qc : :class:`qiskit.QuantumCircuit`
        Quantum Circuit.
    a : int
        Index of the first control qbit.
    b : int
        Index of the second control qbit.
    c : int
        Index of the target qbit.
    """
    
    # a simplified implementation of
    # qc.cx(b, c)
    # qc.cz(a, c)
    # qc.ch(b, c)
    # by combining single-qbit operations
    qc.u(np.pi / 2, np.pi / 4, np.pi, c)
    qc.cx(b, c)
    qc.tdg(c)
    qc.cx(a, c)
    qc.t(c)
    qc.cx(b, c)
    qc.u(np.pi / 2, 0, 3 * np.pi / 4, c)

def comp_add(qc, bs, ss, cs):
    """Function to compute 4-bit controlled addition.

    Parameters
    ----------
    qc : :class:`qiskit.QuantumCircuit`
        Quantum Circuit.
    bs : list
        Indices of the 4 control qbits.
    ss : list
        Indices of the 2 sum qbits.
    cs : int
        Indices of the 3 ancilla qbits.
    """

    # compute multi-controlled X 
    rccx(qc, bs[0], bs[1], cs[0])
    rccx(qc, cs[0], bs[2], cs[1])
    rccx(qc, cs[1], bs[3], cs[2])
    # compute sum
    rccx(qc, cs[2], ss[0], ss[1])
    qc.cx(cs[2], ss[0])
    # uncompute multi-controlled X
    rccx(qc, cs[1], bs[3], cs[2])
    rccx(qc, cs[0], bs[2], cs[1])
    rccx(qc, bs[0], bs[1], cs[0])

def uncomp_add(qc, bs, ss, cs):
    """Function to uncompute 4-bit controlled addition.

    Parameters
    ----------
    qc : :class:`qiskit.QuantumCircuit`
        Quantum Circuit.
    bs : list
        Indices of the 4 control qbits.
    ss : list
        Indices of the 2 sum qbits.
    cs : int
        Indices of the 3 ancilla qbits.
    """

    # compute multi-controlled X
    rccx(qc, bs[0], bs[1], cs[0])
    rccx(qc, cs[0], bs[2], cs[1])
    rccx(qc, cs[1], bs[3], cs[2])
    # uncompute sum
    qc.cx(cs[2], ss[0])
    rccx(qc, cs[2], ss[0], ss[1])
    # uncompute multi-controlled X
    rccx(qc, cs[1], bs[3], cs[2])
    rccx(qc, cs[0], bs[2], cs[1])
    rccx(qc, bs[0], bs[1], cs[0])

def get_diffuser_gate(num):   
    """Function to implement a general diffuser gate.

    Parameters
    ----------
    num : int
        Number of superposed states.

    Returns
    -------
    D : :class:`qiskit.circuit.Gate`
        Quantum gate representing the circuit.
    """

    # num-bit register for the superposed qbits
    q_s = QuantumRegister(num)
    # (num - 2)-bit register for the ancilla qbits
    q_a = QuantumRegister(num - 3)
    # quantum circuit
    qc = QuantumCircuit(q_s, q_a)

    # apply H and X gates to all qbits
    for i in range(num):
        qc.h(i)
        qc.x(i)

    # implement controlled-Z gate
    # apply H gate to the final qbit
    qc.h(num - 1)
    # flip the final bit
    qc.mct(list(range(num - 1)), num - 1, ancilla_qubits=q_a, mode='v-chain')
    # apply H gate to the final qbit
    qc.h(num - 1)

    # apply X and H gates to all qbits
    for i in range(num):
        qc.x(i)
        qc.h(i)

    # return circuit as gate
    return qc.to_gate()

## The Oracle

In [3]:
def comp_zeros(qc):
    """Function to compute the presence of zeros in a row or column.

    Returns
    -------
    qc : :class:`qiskit.QuantumCircuit`
        Quantum circuit for the oracle.
    """

    # flip all elements to calculate zeros
    for r in range(4):
        for c in range(4):
            qc.x(4 + r * 4 + c)
    # compute zeros
    for i in range(8):
        bs = list(range(4 + i * 4, 4 + (i + 1) * 4))
        if i >= 4:
            bs = [4 + 0 * 4 + i - 4, 4 + 1 * 4 + i - 4, 4 + 2 * 4 + i - 4, 4 + 3 * 4 + i - 4]
        # compute addition
        comp_add(qc, bs, [20, 21], [24, 25, 26])

def uncomp_zeros(qc):
    """Function to uncompute the presence of zeros in a row or column.

    Returns
    -------
    qc : :class:`qiskit.QuantumCircuit`
        Quantum circuit for the oracle.
    """

    # uncompute zeros
    for i in range(8):
        bs = list(range(4 + i * 4, 4 + (i + 1) * 4))
        if i >= 4:
            bs = [4 + 0 * 4 + i - 4, 4 + 1 * 4 + i - 4, 4 + 2 * 4 + i - 4, 4 + 3 * 4 + i - 4]
        # uncompute addition
        uncomp_add(qc, bs, [20, 21], [24, 25, 26])
    # flip back
    for r in range(4):
        for c in range(4):
            qc.x(4 + r * 4 + c)

def comp_ones(qc):
    """Function to compute the presence of two ones in a row or column without other ones in the corresponding columns or rows.

    Returns
    -------
    qc : :class:`qiskit.QuantumCircuit`
        Quantum circuit for the oracle.
    """

    # for rows
    for r in range(4):
        for c in range(4):
            for rx in range(4):
                if rx != r:
                    qc.x(4 + rx * 4 + c)
            bs = [4 + 0 * 4 + c, 4 + 1 * 4 + c, 4 + 2 * 4 + c, 4 + 3 * 4 + c]
            # compute addition
            comp_add(qc, bs, [20, 21], [24, 25, 26])

        # flag if sum is 10 or 11
        qc.cx(21, 22)

        for c in range(4):
            bs = [4 + 0 * 4 + c, 4 + 1 * 4 + c, 4 + 2 * 4 + c, 4 + 3 * 4 + c]
            # uncompute addition
            uncomp_add(qc, bs, [20, 21], [24, 25, 26])
            for rx in range(4):
                if rx != r:
                    qc.x(4 + rx * 4 + c)

    # for columns
    for c in range(4):
        for r in range(4):
            for cx in range(4):
                if cx != c:
                    qc.x(4 + r * 4 + cx)
            bs = [4 + r * 4 + 0, 4 + r * 4 + 1, 4 + r * 4 + 2, 4 + r * 4 + 3]
            # compute addition
            comp_add(qc, bs, [20, 21], [24, 25, 26])

        # flag if sum is 10 or 11
        qc.cx(21, 23)

        for r in range(4):
            bs = [4 + r * 4 + 0, 4 + r * 4 + 1, 4 + r * 4 + 2, 4 + r * 4 + 3]
            # uncompute addition
            uncomp_add(qc, bs, [20, 21], [24, 25, 26])
            for cx in range(4):
                if cx != c:
                    qc.x(4 + r * 4 + cx)

def comp_oracle(qc, q_f):
    """Function to implement the oracle for 3-laser solutions.

    Returns
    -------
    qc : :class:`qiskit.QuantumCircuit`
        Quantum circuit for the oracle.
    q_f : int
        Flag qbit.
    """

    # compute solvability for two lonely ones in a single row/column
    comp_ones(qc)

    # compute presence of zeros
    comp_zeros(qc)

    # if all conditions are met
    qc.x(20)
    qc.x(21)
    qc.x(22)
    qc.x(23)
    qc.mct([20, 21, 22, 23], q_f, ancilla_qubits=[24, 25], mode='v-chain')
    qc.x(23)
    qc.x(22)
    qc.x(21)
    qc.x(20)
    
    # uncompute presence
    uncomp_zeros(qc)

    # uncompute solvability
    comp_ones(qc)

## The QRAM

In [4]:
def init_qRAM(qc, boards):
    """Function to implement qRAM.

    Parameters
    ----------
    qc : :class:`qiskit.QuantumCircuit`
        Quantum Circuit.
    boards : list
        Sequences of asteroid positions.
    """

    ## board #00
    for i in range(4):
        qc.x(i)
    rccx(qc, 0, 1, 24)
    rccx(qc, 24, 2, 25)
    rccx(qc, 25, 3, 26)
    # populate
    for e in range(6):
        qc.cx(26, 4 + 4 * int(boards[0][e][0]) + int(boards[0][e][1]))

    ## board #01
    rccx(qc, 25, 3, 26)
    qc.x(3)
    rccx(qc, 25, 3, 26)
    # populate
    for e in range(6):
        qc.cx(26, 4 + 4 * int(boards[1][e][0]) + int(boards[1][e][1]))

    ## board #03
    rccx(qc, 25, 3, 26)
    rccx(qc, 24, 2, 25)
    qc.x(2)
    rccx(qc, 24, 2, 25)
    rccx(qc, 25, 3, 26)
    # populate
    for e in range(6):
        qc.cx(26, 4 + 4 * int(boards[3][e][0]) + int(boards[3][e][1]))

    ## board #02
    rccx(qc, 25, 3, 26)
    qc.x(3)
    rccx(qc, 25, 3, 26)
    # populate
    for e in range(6):
        qc.cx(26, 4 + 4 * int(boards[2][e][0]) + int(boards[2][e][1]))

    ## board #06
    rccx(qc, 25, 3, 26)
    rccx(qc, 24, 2, 25)
    rccx(qc, 0, 1, 24)
    qc.x(1)
    rccx(qc, 0, 1, 24)
    rccx(qc, 24, 2, 25)
    rccx(qc, 25, 3, 26)
    # populate
    for e in range(6):
        qc.cx(26, 4 + 4 * int(boards[6][e][0]) + int(boards[6][e][1]))

    ## board #07
    rccx(qc, 25, 3, 26)
    qc.x(3)
    rccx(qc, 25, 3, 26)
    # populate
    for e in range(6):
        qc.cx(26, 4 + 4 * int(boards[7][e][0]) + int(boards[7][e][1]))

    ## board #05
    rccx(qc, 25, 3, 26)
    rccx(qc, 24, 2, 25)
    qc.x(2)
    rccx(qc, 24, 2, 25)
    rccx(qc, 25, 3, 26)
    # populate
    for e in range(6):
        qc.cx(26, 4 + 4 * int(boards[5][e][0]) + int(boards[5][e][1]))

    ## board #04
    rccx(qc, 25, 3, 26)
    qc.x(3)
    rccx(qc, 25, 3, 26)
    # populate
    for e in range(6):
        qc.cx(26, 4 + 4 * int(boards[4][e][0]) + int(boards[4][e][1]))
    
    ## board #12
    rccx(qc, 25, 3, 26)
    rccx(qc, 24, 2, 25)
    rccx(qc, 0, 1, 24)
    qc.x(0)
    rccx(qc, 0, 1, 24)
    rccx(qc, 24, 2, 25)
    rccx(qc, 25, 3, 26)
    # populate
    for e in range(6):
        qc.cx(26, 4 + 4 * int(boards[12][e][0]) + int(boards[12][e][1]))

    ## board #13
    rccx(qc, 25, 3, 26)
    qc.x(3)
    rccx(qc, 25, 3, 26)
    # populate
    for e in range(6):
        qc.cx(26, 4 + 4 * int(boards[13][e][0]) + int(boards[13][e][1]))

    ## board #15
    rccx(qc, 25, 3, 26)
    rccx(qc, 24, 2, 25)
    qc.x(2)
    rccx(qc, 24, 2, 25)
    rccx(qc, 25, 3, 26)
    # populate
    for e in range(6):
        qc.cx(26, 4 + 4 * int(boards[15][e][0]) + int(boards[15][e][1]))

    ## board #14
    rccx(qc, 25, 3, 26)
    qc.x(3)
    rccx(qc, 25, 3, 26)
    # populate
    for e in range(6):
        qc.cx(26, 4 + 4 * int(boards[14][e][0]) + int(boards[14][e][1]))

    ## board #10
    rccx(qc, 25, 3, 26)
    rccx(qc, 24, 2, 25)
    rccx(qc, 0, 1, 24)
    qc.x(1)
    rccx(qc, 0, 1, 24)
    rccx(qc, 24, 2, 25)
    rccx(qc, 25, 3, 26)
    # populate
    for e in range(6):
        qc.cx(26, 4 + 4 * int(boards[10][e][0]) + int(boards[10][e][1]))

    ## board #11
    rccx(qc, 25, 3, 26)
    qc.x(3)
    rccx(qc, 25, 3, 26)
    # populate
    for e in range(6):
        qc.cx(26, 4 + 4 * int(boards[11][e][0]) + int(boards[11][e][1]))

    ## board #09
    rccx(qc, 25, 3, 26)
    rccx(qc, 24, 2, 25)
    qc.x(2)
    rccx(qc, 24, 2, 25)
    rccx(qc, 25, 3, 26)
    # populate
    for e in range(6):
        qc.cx(26, 4 + 4 * int(boards[9][e][0]) + int(boards[9][e][1]))

    ## board #08
    rccx(qc, 25, 3, 26)
    qc.x(3)
    rccx(qc, 25, 3, 26)
    # populate
    for e in range(6):
        qc.cx(26, 4 + 4 * int(boards[8][e][0]) + int(boards[8][e][1]))

    # reset
    rccx(qc, 25, 3, 26)
    rccx(qc, 24, 2, 25)
    rccx(qc, 0, 1, 24)
    for i in range(1, 4):
        qc.x(i)

In [5]:
# function
def week3_ans_func(boards):

    # register for 16 board number qbits
    q_b = QuantumRegister(4, 'q_b')
    # register for 16 board position qbits
    q_p = QuantumRegister(16, 'q_p')
    # register for sum and ancilla qbits
    q_s = QuantumRegister(4, 'q_s')
    q_a = QuantumRegister(3, 'q_a')
    # register for flag qbit
    q_f = QuantumRegister(1, 'q_f')
    # classical register to measure solution
    c = ClassicalRegister(4, 'c')
    qc = QuantumCircuit(q_b, q_p, q_s, q_a, q_f, c)

    # initialize board qbits
    for i in range(4):
        qc.h(i)

    # initialize board qbits
    qc.x(q_f)
    qc.h(q_f)
    
    # initialize qRAM
    init_qRAM(qc, boards)
    # compute oracle for unsolvable board
    comp_oracle(qc, q_f)
    # uninitialize qRAM
    init_qRAM(qc, boards)

    # diffuser for boards
    qc.append(get_diffuser_gate(4), list(range(4)) + [24])

    # measure
    qc.measure(list(range(4)), c)

    # return reversed bits for same endian
    return qc.reverse_bits()

## The Quantum Circuit

In [6]:
# sample sequences and solutions
Q = [[  [['0', '2'], ['1', '0'], ['1', '2'], ['1', '3'], ['2', '0'], ['3', '3']],
        [['0', '0'], ['0', '1'], ['1', '2'], ['2', '2'], ['3', '0'], ['3', '3']],
        [['0', '0'], ['1', '1'], ['1', '3'], ['2', '0'], ['3', '2'], ['3', '3']],
        [['0', '0'], ['0', '1'], ['1', '1'], ['1', '3'], ['3', '2'], ['3', '3']],
        [['0', '2'], ['1', '0'], ['1', '3'], ['2', '0'], ['3', '2'], ['3', '3']],
        [['1', '1'], ['1', '2'], ['2', '0'], ['2', '1'], ['3', '1'], ['3', '3']],
        [['0', '2'], ['0', '3'], ['1', '2'], ['2', '0'], ['2', '1'], ['3', '3']],
        [['0', '0'], ['0', '3'], ['1', '2'], ['2', '2'], ['2', '3'], ['3', '0']],
        [['0', '3'], ['1', '1'], ['1', '2'], ['2', '0'], ['2', '1'], ['3', '3']],
        [['0', '0'], ['0', '1'], ['1', '3'], ['2', '1'], ['2', '3'], ['3', '0']],
        [['0', '1'], ['0', '3'], ['1', '2'], ['1', '3'], ['2', '0'], ['3', '2']],
        [['0', '0'], ['1', '3'], ['2', '0'], ['2', '1'], ['2', '3'], ['3', '1']],
        [['0', '1'], ['0', '2'], ['1', '0'], ['1', '2'], ['2', '2'], ['2', '3']],
        [['0', '3'], ['1', '0'], ['1', '3'], ['2', '1'], ['2', '2'], ['3', '0']],
        [['0', '2'], ['0', '3'], ['1', '2'], ['2', '3'], ['3', '0'], ['3', '1']],
        [['0', '1'], ['1', '0'], ['1', '2'], ['2', '2'], ['3', '0'], ['3', '1']]    ],
    [   [['0', '1'], ['0', '2'], ['1', '0'], ['2', '0'], ['3', '1'], ['3', '3']],
        [['0', '2'], ['0', '3'], ['1', '1'], ['1', '3'], ['2', '0'], ['2', '1']],
        [['0', '0'], ['0', '3'], ['2', '1'], ['2', '2'], ['3', '0'], ['3', '1']],
        [['0', '0'], ['0', '1'], ['0', '2'], ['1', '1'], ['2', '0'], ['3', '2']],
        [['0', '1'], ['1', '2'], ['1', '3'], ['2', '0'], ['3', '0'], ['3', '1']],
        [['0', '2'], ['0', '3'], ['1', '1'], ['2', '0'], ['2', '1'], ['3', '0']],
        [['0', '0'], ['0', '3'], ['1', '2'], ['2', '0'], ['2', '1'], ['3', '3']],
        [['0', '2'], ['1', '1'], ['1', '3'], ['2', '0'], ['2', '3'], ['3', '2']],
        [['0', '1'], ['0', '3'], ['2', '0'], ['2', '2'], ['3', '0'], ['3', '3']],
        [['0', '0'], ['0', '2'], ['1', '0'], ['2', '2'], ['2', '3'], ['3', '3']],
        [['1', '0'], ['1', '3'], ['2', '1'], ['2', '2'], ['3', '2'], ['3', '3']],
        [['0', '0'], ['1', '0'], ['2', '1'], ['2', '2'], ['3', '2'], ['3', '3']],
        [['0', '0'], ['1', '1'], ['1', '2'], ['2', '1'], ['2', '3'], ['3', '0']],
        [['0', '1'], ['0', '3'], ['2', '1'], ['2', '2'], ['3', '0'], ['3', '1']],
        [['0', '0'], ['0', '1'], ['1', '1'], ['1', '3'], ['3', '2'], ['3', '3']],
        [['0', '0'], ['0', '3'], ['1', '2'], ['1', '3'], ['3', '0'], ['3', '1']]    ]]

# selected sequences of light states
boards = Q[0]

# get the quantum circuit
qc = week3_ans_func(boards)

# # display
# qc.draw(output='mpl')

## Simulation

In [7]:
# execute the quantum circuit on the backend
backend = Aer.get_backend('qasm_simulator')
job = execute(qc, backend, shots=1000)

# obtain the maximum from the result
result = job.result()
result.get_counts()

{'0000': 28,
 '0001': 31,
 '0010': 38,
 '0011': 38,
 '0100': 30,
 '0101': 37,
 '0110': 36,
 '0111': 40,
 '1000': 42,
 '1001': 41,
 '1010': 457,
 '1011': 41,
 '1100': 30,
 '1101': 37,
 '1110': 35,
 '1111': 39}

## Grading

In [8]:
# dependencies for calculating cost
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import Unroller

# calculate cost
ur = Unroller(['u3', 'cx'])
pm = PassManager(ur)
qc_cost = pm.run(qc) 
op_dict = qc_cost.count_ops()
# given criteria to calculate cost
print('Cost: {}'.format(op_dict['u3'] + op_dict['cx'] * 10))

Cost: 42446
